# Grid2x2 using RLlib

In [1]:
import csv
import os

import ray
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility
from ray.tune.registry import register_env

from envs import MultiAgentSumoEnv
from observation import Grid2x2ObservationFunction
from reward_functions import combined_reward

In [2]:
import random
import numpy as np
import torch

TEST_NUM = 7
SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ENV_NAME = "grid2x2"
assert not os.path.exists(os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}"))

In [3]:
def train_env_creator(args):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(**env_params)
    return env

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

train_env = MultiAgentEnvCompatibility(train_env_creator({}))

config: PPOConfig
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_num_workers=1,
        evaluation_sample_timeout_s=300,
    )
    .debugging(seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "1")))
    .multi_agent(
        policies=set(train_env.env.ts_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .fault_tolerance(recreate_failed_workers=True)
)

delta_wait_time_reward 0.85


In [5]:
csv_dir = os.path.join("outputs",ENV_NAME,f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

## Play Untrained Agent

In [6]:
def eval_env_creator(csv_path, tb_log_dir):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(eval=True, csv_path=csv_path, tb_log_dir=tb_log_dir, **env_params)
    return env

In [7]:
ray.init()

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_untrained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure"])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

2023-06-05 01:19:38,558	INFO worker.py:1625 -- Started a local Ray instance.


In [8]:
algo = config.build()

2023-06-05 01:19:42,093	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-05 01:19:42,124	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=29500) delta_wait_time_reward 0.85
(RolloutWorker pid=1940) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=20032) 2023-06-05 01:19:48,881	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-05 01:19:52,570	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=7384) delta_wait_time_reward 0.85 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=25552) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 9x across cluster]


2023-06-05 01:19:56,586	INFO trainable.py:172 -- Trainable.setup took 14.466 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(RolloutWorker pid=7384) 2023-06-05 01:19:56,500	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


In [9]:
algo.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~231000.00UPS, TraCI: 251ms, vehicles TOT 1715 ACT 231 BT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~231000.00UPS, TraCI: 251ms, vehicles TOT 1715 ACT 231 BT 3 ACT 3 BUF 0)                      


{'evaluation': {'episode_reward_max': -315584.44942888705,
  'episode_reward_min': -315584.44942888705,
  'episode_reward_mean': -315584.44942888705,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -75579.12950596321,
   '2': -82284.05196430066,
   '5': -78189.73550824964,
   '6': -79531.53245037395},
  'policy_reward_max': {'1': -75579.12950596321,
   '2': -82284.05196430066,
   '5': -78189.73550824964,
   '6': -79531.53245037395},
  'policy_reward_mean': {'1': -75579.12950596321,
   '2': -82284.05196430066,
   '5': -78189.73550824964,
   '6': -79531.53245037395},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-315584.44942888705],
   'episode_lengths': [720],
   'policy_1_reward': [-75579.12950596321],
   'policy_2_reward': [-82284.05196430066],
   'policy_5_reward': [-78189.73550824964],
   'policy_6_reward': [-79531.53245037395]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 4.280402491724276,
   'mean_inf

In [10]:
ray.shutdown()

## Train RL Agent

In [11]:
ray.init()

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(train_env_creator(config)))

2023-06-05 01:24:03,455	INFO worker.py:1625 -- Started a local Ray instance.


In [12]:
algo = config.build()

2023-06-05 01:24:06,762	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=19592) delta_wait_time_reward 0.85


(RolloutWorker pid=19936) 2023-06-05 01:24:13,290	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


(RolloutWorker pid=19936) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      


2023-06-05 01:24:13,630	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-05 01:24:17,616	INFO trainable.py:172 -- Trainable.setup took 10.848 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [13]:
from datetime import datetime

TRAIN_EPS = 1400  # 720 * 1400 == 1_008_000 total timesteps
CHECKPOINT_FREQ = 100
assert TRAIN_EPS % CHECKPOINT_FREQ == 0

tic = datetime.now()

for i in range(TRAIN_EPS):
    results = algo.train()

    if (i+1) % CHECKPOINT_FREQ == 0:
        algo.save(os.path.join("ray_checkpoints","grid2x2",f"test_{TEST_NUM}"))

toc = datetime.now()

2023-06-05 01:24:31,507	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 86ms, vehicles TOT 2097 ACT 87 BUF 9)               OT 3 ACT 3 BUF 0)                      
(RolloutWorker pid=19204) delta_wait_time_reward 0.85 [repeated 10x across cluster]
(RolloutWorker pid=19204) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]
Step #3600.00 (1ms ~= 1000.00*RT, ~130000.00UPS, TraCI: 99ms, vehicles TOT 2077 ACT 130 BUT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~142000.00UPS, TraCI: 101ms, vehicles TOT 2085 ACT 142 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~204000.00UPS, TraCI: 122ms, vehicles TOT 1543 ACT 204 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~205000.00UPS, TraCI: 123ms, vehicles TOT 1750 ACT 205 B28ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~217000.00UPS, TraCI: 104ms, vehicles TOT 1763 ACT 217 BOT 3 ACT 3 BUF 0)              

(RolloutWorker pid=3172) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=3172) Quitting (on error).
(RolloutWorker pid=19204) Quitting (on error).
(RolloutWorker pid=3172) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 50879 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=3172)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 88ms, vehicles TOT 2106 ACT 51 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 93ms, vehicles TOT 2106 ACT 49 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~49000.00UPS, TraCI: 83ms, vehicles TOT 2106 ACT 49 BUF 8ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~50000.00UPS, TraCI: 85ms, vehicles TOT 2106 ACT 50 BUF 22ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~53000.00UPS, TraCI: 70ms, vehicles TOT 2106 ACT 53 BUF 16ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~58000.00UPS, TraCI: 62ms, vehicles TOT 2106 ACT 58 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 65ms, vehicles TOT 2106 ACT 45 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 73ms, vehicles TOT 2106 ACT 46 BUF 0)               OT

(RolloutWorker pid=15836) 2023-06-05 03:33:27,843	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=15836) Traceback (most recent call last):
(RolloutWorker pid=15836)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=15836)     return func(self, *args, **kwargs)
(RolloutWorker pid=15836)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=15836)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=15836)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=15836)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=15836)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

...

(RolloutWorker pid=8564) Could not connect to TraCI server at localhost:54021 [WinError 10061] No connection could be made because the target machine actively refused it


2023-06-05 05:52:17,280	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff0d29d354112a2fd5051af20501000000 Worker ID: c825ebfb5ac50929be46539d876a77ab7ad391e1af5ec65243e27285 Node ID: a5e0dce584bf0465ade804adea9f6be1007fbf09953e41b27f21a962 Worker IP address: 127.0.0.1 Worker port: 49798 Worker PID: 8564 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.
(RolloutWorker pid=8564) 2023-06-05 05:52:17,275	ERROR worker.py:844 -- Worker exits with an exit code 1.
(RolloutWorker pid=8564) Traceback (most recent call last):
(RolloutWorker pid=8564)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=8564)     return func(self, *args, **kwargs)
(RolloutWorker pid=8564)   File "c:\Users\admin\anaconda3\env

(RolloutWorker pid=23192) delta_wait_time_reward 0.85
(RolloutWorker pid=23192) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 43 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 68ms, vehicles TOT 2106 ACT 40 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~42000.00UPS, TraCI: 69ms, vehicles TOT 2106 ACT 42 BUF 25ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~47000.00UPS, TraCI: 87ms, vehicles TOT 2106 ACT 47 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 80ms, vehicles TOT 2106 ACT 41 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 80ms, vehicles TOT 2106 ACT 43 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~46000.00UPS, TraC

(RolloutWorker pid=29588) 2023-06-05 06:06:49,937	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=29588) Traceback (most recent call last):
(RolloutWorker pid=29588)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=29588)     return func(self, *args, **kwargs)
(RolloutWorker pid=29588)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=29588)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=29588)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=29588)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=29588)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

(RolloutWorker pid=424) delta_wait_time_reward 0.85
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 75ms, vehicles TOT 2106 ACT 41 BUF 0)               24ms, vehicles TOT 3 ACT 3 BUF 0)      
(RolloutWorker pid=424) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 80ms, vehicles TOT 2106 ACT 50 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 69ms, vehicles TOT 2106 ACT 43 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~43000.00UPS, TraCI: 65ms, vehicles TOT 2106 ACT 43 BUF 15ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 79ms, vehicles TOT 2106 ACT 40 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 74ms, vehicles TOT 2106 ACT 45 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 74ms, vehicles TO

In [14]:
str(toc - tic)

'5:28:58.090030'

In [15]:
ray.shutdown()

## Play Trained Agent

In [ ]:
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_trained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure"])

In [16]:
register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

In [17]:
ray.init()

2023-06-05 06:53:20,405	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.4.0


In [18]:
from ray.rllib.algorithms.ppo import PPO

checkpoint_path = os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}",f"checkpoint_{TRAIN_EPS:06}")
checkpoint_path = os.path.abspath(checkpoint_path)
print(checkpoint_path)

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-06-05 06:53:23,997	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_7\checkpoint_001400! Trying to extract checkpoint info from other files found in that dir.
2023-06-05 06:53:24,022	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_7\checkpoint_001400
(RolloutWorker pid=21568) delta_wait_time_reward 0.85
(RolloutWorker pid=21568) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=27096) 2023-06-05 06:53:30,851	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-05 06:53:31,104	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-05 06:53:35,251	INFO trainable.py:172 -- Trainable.setup took 11.226 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [20]:
ppo_agent.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 163ms, vehicles TOT 2106 ACT 45 BUFOT 3 ACT 3 BUF 0)                      
(RolloutWorker pid=29284) delta_wait_time_reward 0.85 [repeated 10x across cluster]
(RolloutWorker pid=29284) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]


{'evaluation': {'episode_reward_max': -76934.74607057456,
  'episode_reward_min': -76934.74607057456,
  'episode_reward_mean': -76934.74607057456,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -19229.96091845558,
   '2': -18779.31926716052,
   '5': -19191.235200160903,
   '6': -19734.23068479743},
  'policy_reward_max': {'1': -19229.96091845558,
   '2': -18779.31926716052,
   '5': -19191.235200160903,
   '6': -19734.23068479743},
  'policy_reward_mean': {'1': -19229.96091845558,
   '2': -18779.31926716052,
   '5': -19191.235200160903,
   '6': -19734.23068479743},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-76934.74607057456],
   'episode_lengths': [720],
   'policy_1_reward': [-19229.96091845558],
   'policy_2_reward': [-18779.31926716052],
   'policy_5_reward': [-19191.235200160903],
   'policy_6_reward': [-19734.23068479743]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 4.243369241362637,
   'mean_inf

In [21]:
ray.shutdown()

(RolloutWorker pid=29284) 2023-06-05 06:53:35,138	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
